In [81]:
import numpy as np
import pandas as pd
import awkward as ak
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import optim
import matplotlib.pyplot as plt
import topcoffea.modules.utils as utils
import pickle
import gzip
import logging
import time

In [3]:
fSMEFT = "/afs/crc.nd.edu/user/h/hnelson2/dctr/analysis/dctr_SMEFT.pkl.gz"
# fpowheg = "/afs/crc.nd.edu/user/h/hnelson2/dctr/analysis/dctr_powheg.pkl.gz" 

In [4]:
rando=1234

In [28]:
inputs_smeft= pickle.load(gzip.open(fSMEFT)).get()
smeft_nevents = inputs_smeft.shape[0]

In [57]:
inputs_smeft.shape

(1958489, 11)

In [32]:
# inputs_powheg = pickle.load(gzip.open(fpowheg)).get()
inputs_powheg = ((pickle.load(gzip.open(fSMEFT)).get()).sample(frac=1, random_state=rando).reset_index(drop=True)).iloc[:smeft_nevents]

In [34]:
smeft_train = inputs_smeft.sample(frac=0.7, random_state=rando)
smeft_test = inputs_smeft.drop(smeft_train.index)

In [35]:
powheg_train = inputs_powheg.sample(frac=0.7, random_state=rando)
powheg_test = inputs_powheg.drop(powheg_train.index)

In [36]:
truth_smeft = np.ones_like(smeft_train['weights'])
truth_powheg = np.zeros_like(powheg_train['weights'])

weights_smeft = np.ones_like(smeft_train['weights'])
weights_powheg = np.ones_like(powheg_train['weights'])

In [37]:
z = np.concatenate([smeft_train, powheg_train], axis=0).astype(np.float32)
w = np.concatenate([weights_smeft, weights_powheg], axis=0).astype(np.float32)
y = np.concatenate([truth_smeft, truth_powheg], axis=0).astype(np.float32)

In [71]:
# t_test = torch.from_numpy(np.concatenate([[inputs_smeft['avg_top_pt'].to_numpy()],
#                             [inputs_smeft['mtt'].to_numpy()],
#                             [inputs_smeft['top1pt'].to_numpy()],
#                             [inputs_smeft['top1eta'].to_numpy()],
#                             [inputs_smeft['top1phi'].to_numpy()],
#                             [inputs_smeft['top1mass'].to_numpy()],
#                             [inputs_smeft['top2pt'].to_numpy()],
#                             [inputs_smeft['top2eta'].to_numpy()],
#                             [inputs_smeft['top2phi'].to_numpy()],
#                             [inputs_smeft['top2mass'].to_numpy()],
#                                 ]).T)

t_test = torch.from_numpy(z)

In [72]:
train_dataloader = DataLoader(t_test, batch_size=128)

In [74]:
input_dim = t_test.shape[1]

In [79]:
device = "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.main_module= nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(),
            nn.Linear(256,256),
            nn.LeakyReLU(),
            nn.Linear(256, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.main_module(x)

model = NeuralNetwork(input_dim).to(device)
# print(model)

Using cpu device


In [80]:
loss_fn =  nn.BCELoss(reduction='mean')

In [83]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [84]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [85]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [86]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    # test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


ValueError: too many values to unpack (expected 2)